<a href="https://colab.research.google.com/github/dishakhanted/QA_for_farmers/blob/main/Q%26A_for_farmers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np 
import string
import random

import nltk
from nltk.corpus import brown
from nltk.corpus import reuters

from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

from nltk.corpus import stopwords

from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
import sys
sys.path.append("/content/drive/My Drive/Colab Notebooks/Q&A_for_farmers")
path='/content/drive/My Drive/Colab Notebooks/Q&A_for_farmers'

In [ ]:
result= pd.read_csv(path+'/datadelloite.csv', encoding='utf-8', header=0)
result=result['QueryText']

In [ ]:
#tokenize words in all DOCS 
alldocslist=result
plot_data = [[]] * len(alldocslist)
index=len(alldocslist)-1
for doc in alldocslist:
    text = doc
    tokentext = word_tokenize(text)
    plot_data[index].append(tokentext)
    
    
print(plot_data[1220][1])

['Asking', 'about', 'weather', 'forecast', '?']


In [ ]:
# Navigation: first index gives all documents, second index gives specific document, third index gives words of that doc
plot_data[20][1][0:10]

['Asking', 'about', 'weather', 'forecast', '?']

In [ ]:
#make all words lower case for all docs 
for x in range(len(alldocslist)):
    lowers = [word.lower() for word in plot_data[0][x]]
    plot_data[0][x] = lowers

plot_data[0][1][0:10]

['asking', 'about', 'weather', 'forecast', '?']

In [ ]:
# remove stop words from all docs 
stop_words = set(stopwords.words('english'))

for x in range(len(alldocslist)):
    filtered_sentence = [w for w in plot_data[0][x] if not w in stop_words]
    plot_data[0][x] = filtered_sentence

plot_data[0][1][0:10]

['asking', 'weather', 'forecast', '?']

In [ ]:
#stem words EXAMPLE (could try others/lemmers )

snowball_stemmer = SnowballStemmer("english")
stemmed_sentence = [snowball_stemmer.stem(w) for w in filtered_sentence]
stemmed_sentence[0:10]

porter_stemmer = PorterStemmer()
snowball_stemmer = SnowballStemmer("english")
stemmed_sentence = [ porter_stemmer.stem(w) for w in filtered_sentence]
stemmed_sentence[0:10]

['attack', 'leaf', 'eat', 'caterpillar', '?']

In [ ]:
# Create inverse index which gives document number for each document and where word appears

#first we need to create a list of all words 
l = plot_data[0]
flatten = [item for sublist in l for item in sublist]
words = flatten
wordsunique = set(words)
wordsunique = list(wordsunique)

In [ ]:
# create functions for TD-IDF / BM25
import math
#from textblob import TextBlob as tb

def tf(word, doc):
    return doc.count(word) / len(doc)

def n_containing(word, doclist):
    return sum(1 for doc in doclist if word in doc)

def idf(word, doclist):
    return math.log(len(doclist) / (0.01 + n_containing(word, doclist)))

def tfidf(word, doc, doclist):
    return (tf(word, doc) * idf(word, doclist))


In [ ]:
# Create dictonary of words
# THIS ONE-TIME INDEXING IS THE MOST PROCESSOR-INTENSIVE STEP AND WILL TAKE TIME TO RUN (BUT ONLY NEEDS TO BE RUN ONCE)
import re
import numpy as np

plottest = plot_data[0][0:1000]

worddic = {}

for doc in plottest:
    for word in wordsunique:
        if word in doc:
            word = str(word)
            index = plottest.index(doc)
            positions = list(np.where(np.array(plottest[index]) == word)[0])
            idfs = tfidf(word,doc,plottest)
            try:
                worddic[word].append([index,positions,idfs])
            except:
                worddic[word] = []
                worddic[word].append([index,positions,idfs])

In [ ]:
# the index creates a dic with each word as a KEY and a list of doc indexs, word positions, and td-idf score as VALUES
worddic['asking']

[[1, [0], 0.1302147811860911],
 [3, [1], 0.07440844639205205],
 [1, [0], 0.1302147811860911],
 [1, [0], 0.1302147811860911],
 [9, [0], 0.08680985412406073],
 [1, [0], 0.1302147811860911],
 [11, [1], 0.1302147811860911],
 [1, [0], 0.1302147811860911],
 [1, [0], 0.1302147811860911],
 [1, [0], 0.1302147811860911],
 [1, [0], 0.1302147811860911],
 [19, [0], 0.10417182494887288],
 [1, [0], 0.1302147811860911],
 [1, [0], 0.1302147811860911],
 [22, [0], 0.06510739059304554],
 [1, [0], 0.1302147811860911],
 [1, [0], 0.1302147811860911],
 [1, [0], 0.1302147811860911],
 [26, [1], 0.08680985412406073],
 [28, [0], 0.1302147811860911],
 [1, [0], 0.1302147811860911],
 [1, [0], 0.1302147811860911],
 [33, [0], 0.06510739059304554],
 [1, [0], 0.1302147811860911],
 [1, [0], 0.1302147811860911],
 [1, [0], 0.1302147811860911],
 [42, [1], 0.07440844639205205],
 [1, [0], 0.1302147811860911],
 [48, [0], 0.1302147811860911],
 [28, [0], 0.1302147811860911],
 [1, [0], 0.1302147811860911],
 [1, [0], 0.13021478118

In [ ]:
# pickel (save) the dictonary to avoid re-calculating
np.save('worddic_1000.npy', worddic)

In [ ]:
from collections import Counter

def search(searchsentence):
    
        # split sentence into individual words 
        searchsentence = searchsentence.lower()
        try:
            words = searchsentence.split(' ')
        except:
            words = list(words)
        enddic = {}
        idfdic = {}
        closedic = {}
        
        # remove words if not in worddic 
        realwords = []
        for word in words:
            if word in list(worddic.keys()):
                realwords.append(word)  
        words = realwords
        numwords = len(words)
        
        # make metric of number of occurances of all words in each doc & largest total IDF 
        for word in words:
            for indpos in worddic[word]:
                index = indpos[0]
                amount = len(indpos[1])
                idfscore = indpos[2]
                enddic[index] = amount
                idfdic[index] = idfscore
                fullcount_order = sorted(enddic.items(), key=lambda x:x[1], reverse=True)
                fullidf_order = sorted(idfdic.items(), key=lambda x:x[1], reverse=True)
        # make metric of what percentage of words appear in each doc
        combo = []
        alloptions = {k: worddic.get(k, None) for k in (words)}
        for worddex in list(alloptions.values()):
            for indexpos in worddex:
                for indexz in indexpos:
                    combo.append(indexz)
                    
        comboindex = combo[::3]
        combocount = Counter(comboindex)
        #print(comboindex)
        for key in combocount:
            combocount[key] = combocount[key] / numwords
        combocount_order = sorted(combocount.items(), key=lambda x:x[1], reverse=True)
        #print(combocount_order)
        # make metric for if words appear in same order as in search
        if len(words) > 1:
            x = []
            y = []
            for record in [worddic[z] for z in words]:
                
                for index in record:
                    #print(index[0])
                    x.append(index[0])
            for i in x:
               # print(i)
                if x.count(i) > 1:
                    y.append(i)
                    #print(i)
            y = list(set(y))

            closedic = {}
            for wordbig in [worddic[x] for x in words]:
                for record in wordbig:
                    if record[0] in y:
                        #print(record[0])
                        index = record[0]
                        positions = record[1]
                        try:
                            closedic[index].append(positions)
                        except:
                            closedic[index] = []
                            closedic[index].append(positions)
            #print(closedic)
            x = 0
            fdic = {}
            for index in y:
                csum = []
                for seqlist in closedic[index]:
                    while x > 0:
                        secondlist = seqlist
                        x = 0
                        sol = [1 for i in firstlist if i + 1 in secondlist]
                        csum.append(sol)
                        fsum = [item for sublist in csum for item in sublist]
                        fsum = sum(fsum)
                        fdic[index] = fsum
                        fdic_order = sorted(fdic.items(), key=lambda x:x[1], reverse=True)
                    while x == 0:
                        firstlist = seqlist
                        x = x + 1
        else:
            fdic_order = 0
                    
        # also the one above should be given a big boost if ALL found together 
           
        
        #could make another metric for if they are not next to each other but still close 
        
        
        return(searchsentence,words,fullcount_order,combocount_order,fullidf_order,fdic_order)
    
    


In [ ]:
def rank(term):
    results = search(term)

    # get metrics 
    num_score = results[2]
    per_score = results[3]
    tfscore = results[4]
    order_score = results[5]
    
    final_candidates = []

    # rule1: if high word order score & 100% percentage terms then put at top position
    try:
        first_candidates = []
        for candidates in order_score:
            if candidates[1] > 1:
                first_candidates.append(candidates[0])

        second_candidates = []

        for match_candidates in per_score:
            if match_candidates[1] == 1:
                second_candidates.append(match_candidates[0])
            if match_candidates[1] == 1 and match_candidates[0] in first_candidates:
                final_candidates.append(match_candidates[0])

    # rule2: next add other word order score which are greater than 1 

        t3_order = first_candidates[0:3]
        for each in t3_order:
            if each not in final_candidates:
                final_candidates.insert(len(final_candidates),each)

    # rule3: next add top td-idf results
        final_candidates.insert(len(final_candidates),tfscore[0][0])
        final_candidates.insert(len(final_candidates),tfscore[1][0])
        # rule4: next add other high percentage score 
        t3_per = second_candidates[0:3]
        for each in t3_per:
            if each not in final_candidates:
                final_candidates.insert(len(final_candidates),each)

    #rule5: next add any other top results for metrics
        othertops = [num_score[0][0],per_score[0][0],tfscore[0][0],order_score[0][0]]
        for top in othertops:
            if top not in final_candidates:
                final_candidates.insert(len(final_candidates),top)
                
    # unless single term searched, in which case just return 
    except:
        othertops = [num_score[0][0],num_score[1][0],num_score[2][0],per_score[0][0],tfscore[0][0]]
        for top in othertops:
            if top not in final_candidates:
                final_candidates.insert(len(final_candidates),top)
    l=[]
    ind=[]
    for index, results in enumerate(final_candidates):
        if index < 3:
            #print("RESULT", index + 1, ":", alldocslist[results])
            l.append(alldocslist[results])
            ind.append(results)
           # print(results)
    return l,ind        
            

In [ ]:
s=['What is the market rate of rice?','Weather forecast of tirora block','crop water requirement','today weather in partur','soil testing information','weed control techniques','what is the fertilizer of wheat','insecticides attack of catterpillar','fertilizer dose for sugercane','protection from attack of grub','protection from attack of wilt','water requirement for fruits and vegetables','how to control rust attack','variety of green gram','information about spray for flower drop control','varieties of papaya','market price of pigeon pea','control of blight attack on tomato','variety of bean','market rate of begal gram']

In [ ]:
#Getting dataset of all the answers of questions asked
j1= pd.read_csv(path+'/datafile.csv', encoding='utf-8', header=0)
j2= pd.read_csv(path+'/datafile 1.csv', encoding='utf-8', header=0)
j3= pd.read_csv(path+'/datafile 2.csv', encoding='utf-8', header=0)
j4= pd.read_csv(path+'/datafile 3.csv', encoding='utf-8', header=0)
j5= pd.read_csv(path+'/datafile 4.csv', encoding='utf-8', header=0)
j6= pd.read_csv(path+'/datafile 5.csv', encoding='utf-8', header=0)
j=pd.concat([j1,j2,j3,j4,j5,j6],axis=0)
j=pd.DataFrame(j)
j=j.reset_index()
j=j['KCCAns']


In [ ]:
#values of features considered to rank the documents in database
d=pd.DataFrame()


for i in range(20):
    term=s[i]
    ret=search(str(term))
    a=pd.DataFrame([ret])
    d= pd.concat([d, a], axis=0, sort=False)
#s=search('weather asking')
#a=pd.DataFrame([s])
#a
#d=d.rename({0:'Rank_1_Query',1:'Rank_2_Query',2:'Rank_3_query',},axis='columns')
d=d.rename({0:'Searched_Query',1:'Searched_Keywords',2:'Num_occur',4:'tf_idf_values',5:'Order_keywords_score'},axis='columns')
d=d.drop(columns=[3])
d

,Searched_Query,Searched_Keywords,Num_occur,tf_idf_values,Order_keywords_score
0,what is the market rate of rice?,"[market, rate]","[(0, 1), (2, 1), (184, 1), (195, 1), (215, 1),...","[(678, 1.2754591276909577), (319, 0.6475334602...","[(834, 2), (880, 2), (0, 1), (2, 1), (265, 1),..."
0,weather forecast of tirora block,"[weather, forecast, tirora, block]","[(1, 1), (3, 1), (55, 1), (57, 1), (62, 1), (6...","[(625, 0.6694713532990454), (3, 0.573832588542...","[(3, 3), (64, 1), (1, 1), (928, 1), (892, 1), ..."
0,crop water requirement,"[crop, water]","[(33, 1), (323, 1), (409, 1), (414, 1), (503, ...","[(249, 1.103792807532732), (898, 1.10379280753...","[(33, 0)]"
0,today weather in partur,[weather],"[(1, 1), (3, 1), (55, 1), (57, 1), (62, 1), (6...","[(1, 0.17029971176102607), (329, 0.17029971176...",0
0,soil testing information,"[soil, testing, information]","[(67, 1), (208, 1), (492, 1), (592, 1), (174, ...","[(174, 1.8396546792212196), (289, 1.1037928075...","[(492, 2), (208, 1), (592, 1), (67, 1)]"
0,weed control techniques,"[weed, control]","[(4, 1), (60, 1), (187, 1), (542, 1), (850, 1)...","[(60, 1.1272378320618872), (542, 1.12723783206...","[(32, 0), (4, 0), (18, 0), (532, 0), (60, 0)]"
0,what is the fertilizer of wheat,"[fertilizer, wheat]","[(186, 1), (233, 1), (259, 1), (263, 1), (362,...","[(770, 1.573161316093246), (878, 1.53944825740...","[(233, 1), (60, 1), (259, 0), (613, 0), (426, ..."
0,insecticides attack of catterpillar,"[attack, catterpillar]","[(5, 1), (9, 1), (11, 1), (19, 1), (22, 1), (3...","[(11, 1.1773550518404727), (526, 1.17735505184...","[(526, 2), (9, 1), (11, 1), (42, 1), (471, 1),..."
0,fertilizer dose for sugercane,"[fertilizer, dose]","[(186, 1), (233, 1), (259, 1), (263, 1), (362,...","[(878, 1.53944825740738), (421, 1.422661306211...","[(263, 2), (821, 2), (421, 1), (454, 1), (613,..."
0,protection from attack of grub,"[attack, grub]","[(5, 1), (9, 1), (11, 1), (19, 1), (22, 1), (3...","[(545, 1.3795609896257939), (204, 0.6345806203...","[(11, 1), (19, 1), (251, 1), (258, 0), (5, 0),..."


In [ ]:
#searched query and their respected answers
ans=[]
y=pd.DataFrame()
for i in range(20):
    ans=[]
    ret2rank,ret2index=rank(str(s[i]))
    for k in ret2index:
        ret2rank.append(j[k])
    x=pd.DataFrame([ret2rank])
    y=pd.concat([y,x])
y
#y.to_csv('answers.csv')
y=y.rename({0:'Rank_1_Query',1:'Rank_2_Query',2:'Rank_3_query',3:'Rank_1_Query_answer',4:'Rank_2_Query_answer',5:'Rank_3_Query_answer'},axis='columns')
y

,Rank_1_Query,Rank_2_Query,Rank_3_query,Rank_1_Query_answer,Rank_2_Query_answer,Rank_3_Query_answer
0,FARMER ASKED ABOUT MARKET RATE OF ?\n,FARMER ASKED ABOUT MARKET RATE OF REDGRAM ?\n,tur market,SOYABEAN AVERAGE MARKET RATE OF -3365-3455 RS...,WASHIM MARKET RATE OF REDGRAM 4550-4700- RS/Q...,WASHIM\tLAL\tQUINTAL\t270\t4200\t4525\t4400
0,Farmer Asking about weather forecast Of Tirora...,Asking about weather forecast Of Malegaon Bloc...,Farmer Asking about weather forecast Of Tirora...,Weather forecast Of Tirora Block- The weather...,Weather forecast Of Malegaon Block -This week ...,Weather forecast Of Tirora Block- The weather...
0,ASKED ABOUT IRRIGATION OF WATER IN GRAM?,asked about water management in chick pea,ASKING ABOUT CONTACT NUMBER FOR PRADHANMANTRI ...,IRRIGATION OF WATER IN GRAM - GIVE WATER TO G...,gram is a very sensitive crop. Green gram requ...,CONTACT NUMBER FOR PRADHANMANTRI CROP INSURANC...
0,Asking about weather forecast?\n,Farmer Asking about weather forecast Of Tirora...,Farmer Asking About Weather Forecast?\n,THIS WEEK WEATHER IS CLOUDY AND NO RAINFALL CH...,Weather forecast Of Tirora Block- The weather...,This week weather is CLOUDY and LIGHT Possibil...
0,TOLD ABOUT SOIL TESTING & SOIL HEALTH CARD,Ask about General information\t,ASKING ABOUT DETAIL INFORMATION ABOUT FISHERIE...,"P.C KVK, Karda, Tq.Risod, Distt.Washim.Ph:0725...",all information provided - thanks for calling ...,Maharashtra Animal and Fishery Sciences Univer...
0,WEED MANAGEMENT IN WHEAT ?\n,WEED MANAGEMENT IN ONION ?\t\n,WEED MANAGEMENT IN BENGAL GRAM ?,SPRAY ALLGRIP (METSULFURON METHYL 20% WG)8 GM ...,SPRAY GOAL 20 ML / 15 LITERS OF WATER 21 DAYS ...,WEED MANAGEMENT IN BENGAL GRAM- SPRAY Pendimet...
0,rust in wheat,FERTILIZER SPRAY \n,FARMER ASKING ABOUT FERTILIZER SPRAY FOR GROWT...,rust in wheat - spray ridomil gold 30 gm / 15...,SPRAY 00 52 34 60 GM + MICROLLA 30 ML /15 LIT...,FERTILIZER SPRAY FOR GROWTH OF WHEAT SPRAY 19 ...
0,ASKING ABOUT ATTACK OF CATTERPILLAR ON ?,INSECTICIDE \nASKING ABOUT ATTACK OF CATTERPIL...,ASKING ABOUT ATTACK OF CATTERPILLAR ON ?,Spray Quinolphos 30 ml/15 liter of water.\n,Spray Proclaim 07 gm/15 liter of Water(Emamect...,Spray Quinolphos 30 ml/15 liter of water.\n
0,FARMER ASKED ABOUT FERTILIZER DOSE FOR MELON?,ASKING ABOUT FERTILIZER DOSE FOR GROUND NUT ?,FERTILIZER SPRAY \n,"50 kg of nitrogen, 50 kg of phosphorus and 50 ...",\nAmmonium sulphate and single super phosphate...,SPRAY 00 52 34 60 GM + MICROLLA 30 ML /15 LIT...
0,Attack of White Grub On Onion ?,WILT ATTACK ?,ASKED ABOUT STEM BORER ATTACK ON PADDY?,\tDrenching of Chloropyriphos 40 ml / 15 lit o...,DRAINCHING OF BLUE COPPER 30 GM + HUMIC ACID 3...,Foliar spray- Mix 5 – 10 ml Varunastra per lit...


In [ ]:
#concatenating both the resulting dataframes i.e. query answers and the methods to rank them
final_data1=pd.concat([d,y],axis=1)
final_data1=final_data1.reset_index()
final_data1=final_data1.drop(columns=['index'])
final_data1

,Searched_Query,Searched_Keywords,Num_occur,tf_idf_values,Order_keywords_score,Rank_1_Query,Rank_2_Query,Rank_3_query,Rank_1_Query_answer,Rank_2_Query_answer,Rank_3_Query_answer
0,what is the market rate of rice?,"[market, rate]","[(0, 1), (2, 1), (184, 1), (195, 1), (215, 1),...","[(678, 1.2754591276909577), (319, 0.6475334602...","[(834, 2), (880, 2), (0, 1), (2, 1), (265, 1),...",FARMER ASKED ABOUT MARKET RATE OF ?\n,FARMER ASKED ABOUT MARKET RATE OF REDGRAM ?\n,tur market,SOYABEAN AVERAGE MARKET RATE OF -3365-3455 RS...,WASHIM MARKET RATE OF REDGRAM 4550-4700- RS/Q...,WASHIM\tLAL\tQUINTAL\t270\t4200\t4525\t4400
1,weather forecast of tirora block,"[weather, forecast, tirora, block]","[(1, 1), (3, 1), (55, 1), (57, 1), (62, 1), (6...","[(625, 0.6694713532990454), (3, 0.573832588542...","[(3, 3), (64, 1), (1, 1), (928, 1), (892, 1), ...",Farmer Asking about weather forecast Of Tirora...,Asking about weather forecast Of Malegaon Bloc...,Farmer Asking about weather forecast Of Tirora...,Weather forecast Of Tirora Block- The weather...,Weather forecast Of Malegaon Block -This week ...,Weather forecast Of Tirora Block- The weather...
2,crop water requirement,"[crop, water]","[(33, 1), (323, 1), (409, 1), (414, 1), (503, ...","[(249, 1.103792807532732), (898, 1.10379280753...","[(33, 0)]",ASKED ABOUT IRRIGATION OF WATER IN GRAM?,asked about water management in chick pea,ASKING ABOUT CONTACT NUMBER FOR PRADHANMANTRI ...,IRRIGATION OF WATER IN GRAM - GIVE WATER TO G...,gram is a very sensitive crop. Green gram requ...,CONTACT NUMBER FOR PRADHANMANTRI CROP INSURANC...
3,today weather in partur,[weather],"[(1, 1), (3, 1), (55, 1), (57, 1), (62, 1), (6...","[(1, 0.17029971176102607), (329, 0.17029971176...",0,Asking about weather forecast?\n,Farmer Asking about weather forecast Of Tirora...,Farmer Asking About Weather Forecast?\n,THIS WEEK WEATHER IS CLOUDY AND NO RAINFALL CH...,Weather forecast Of Tirora Block- The weather...,This week weather is CLOUDY and LIGHT Possibil...
4,soil testing information,"[soil, testing, information]","[(67, 1), (208, 1), (492, 1), (592, 1), (174, ...","[(174, 1.8396546792212196), (289, 1.1037928075...","[(492, 2), (208, 1), (592, 1), (67, 1)]",TOLD ABOUT SOIL TESTING & SOIL HEALTH CARD,Ask about General information\t,ASKING ABOUT DETAIL INFORMATION ABOUT FISHERIE...,"P.C KVK, Karda, Tq.Risod, Distt.Washim.Ph:0725...",all information provided - thanks for calling ...,Maharashtra Animal and Fishery Sciences Univer...
5,weed control techniques,"[weed, control]","[(4, 1), (60, 1), (187, 1), (542, 1), (850, 1)...","[(60, 1.1272378320618872), (542, 1.12723783206...","[(32, 0), (4, 0), (18, 0), (532, 0), (60, 0)]",WEED MANAGEMENT IN WHEAT ?\n,WEED MANAGEMENT IN ONION ?\t\n,WEED MANAGEMENT IN BENGAL GRAM ?,SPRAY ALLGRIP (METSULFURON METHYL 20% WG)8 GM ...,SPRAY GOAL 20 ML / 15 LITERS OF WATER 21 DAYS ...,WEED MANAGEMENT IN BENGAL GRAM- SPRAY Pendimet...
6,what is the fertilizer of wheat,"[fertilizer, wheat]","[(186, 1), (233, 1), (259, 1), (263, 1), (362,...","[(770, 1.573161316093246), (878, 1.53944825740...","[(233, 1), (60, 1), (259, 0), (613, 0), (426, ...",rust in wheat,FERTILIZER SPRAY \n,FARMER ASKING ABOUT FERTILIZER SPRAY FOR GROWT...,rust in wheat - spray ridomil gold 30 gm / 15...,SPRAY 00 52 34 60 GM + MICROLLA 30 ML /15 LIT...,FERTILIZER SPRAY FOR GROWTH OF WHEAT SPRAY 19 ...
7,insecticides attack of catterpillar,"[attack, catterpillar]","[(5, 1), (9, 1), (11, 1), (19, 1), (22, 1), (3...","[(11, 1.1773550518404727), (526, 1.17735505184...","[(526, 2), (9, 1), (11, 1), (42, 1), (471, 1),...",ASKING ABOUT ATTACK OF CATTERPILLAR ON ?,INSECTICIDE \nASKING ABOUT ATTACK OF CATTERPIL...,ASKING ABOUT ATTACK OF CATTERPILLAR ON ?,Spray Quinolphos 30 ml/15 liter of water.\n,Spray Proclaim 07 gm/15 liter of Water(Emamect...,Spray Quinolphos 30 ml/15 liter of water.\n
8,fertilizer dose for sugercane,"[fertilizer, dose]","[(186, 1), (233, 1), (259, 1), (263, 1), (362,...","[(878, 1.53944825740738), (421, 1.42266